In [3]:
import cv2
import sys
import os
import numpy as np
import time

config_file = "./yolov4-tiny-custom.cfg"
#config_file = "./yolov3-tiny.cfg"
data_file = "./obj.data"
weights_file = "./yolov4-tiny-custom_final.weights"
#weights_file = "./yolov3-tiny.weights"
image_path = "./test.jpg"
#image_path ="./img.png"
output_path = "output_image.jpg"
names_path = "./obj.names"
#names_path = "./coco.names"

try:
    net = cv2.dnn.readNet(weights_file, config_file)
except cv2.error as e:
    print(f"Error loading YOLO files: {e}")
    exit()

try:
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
except cv2.error as e:
    print(f"Error getting layer names: {e}")
    exit()

try:
    with open(names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]
except FileNotFoundError:
    print("Error: obj.names file not found")
    exit()
    
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:  # Esc key to stop
        break

cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()